<a href="https://colab.research.google.com/github/AndresT3086/saberPRO-prediction/blob/main/99_modelo_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Predicción Rendimiento Saber Pro
# Notebook con el proceso de generación del archivo de submission

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

import warnings

warnings.filterwarnings('ignore')

In [ ]:
# 1. CARGAR LOS DATOS
print('='*60)
print('PASO 1: CARGA DE DATOS')
print('='*60)

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(f'Train: {train.shape}')
print(f'Test: {test.shape}')
print(f'\nColumnas: {train.shape[1]}')
print(f'Estudiantes en train: {len(train)}')
print(f'Estudiantes en test: {len(test)}')

PASO 1: CARGA DE DATOS
Train: (692500, 21)
Test: (296786, 20)

Columnas: 21
Estudiantes en train: 692500
Estudiantes en test: 296786


In [ ]:
# 2. EXPLORACIÓN RÁPIDA
print('\n' + '='*60)
print('PASO 2: EXPLORACIÓN DE DATOS')
print('='*60)

print('\nDistribución de clases:')
print(train['RENDIMIENTO_GLOBAL'].value_counts())
print('\nPorcentajes:')
print(train['RENDIMIENTO_GLOBAL'].value_counts(normalize=True) * 100)

print(f'\nValores faltantes en train: {train.isnull().sum().sum()}')
print(f'Valores faltantes en test: {test.isnull().sum().sum()}')


PASO 2: EXPLORACIÓN DE DATOS

Distribución de clases:
RENDIMIENTO_GLOBAL
alto          175619
bajo          172987
medio-bajo    172275
medio-alto    171619
Name: count, dtype: int64

Porcentajes:
RENDIMIENTO_GLOBAL
alto          25.360144
bajo          24.980072
medio-bajo    24.877256
medio-alto    24.782527
Name: proportion, dtype: float64

Valores faltantes en train: 297378
Valores faltantes en test: 128614


In [ ]:
# 3. GUARDAR IDs Y SEPARAR VARIABLES
print('\n' + '='*60)
print('PASO 3: PREPARACIÓN DE DATOS')
print('='*60)

test_ids = test['ID'].copy()
y_train = train['RENDIMIENTO_GLOBAL'].copy()
X_train = train.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1)
X_test = test.drop(['ID'], axis=1)

print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}')


PASO 3: PREPARACIÓN DE DATOS
X_train: (692500, 19)
y_train: (692500,)
X_test: (296786, 19)


In [ ]:
# 4. PREPROCESAMIENTO
print('\n' + '='*60)
print('PASO 4: PREPROCESAMIENTO')
print('='*60)

# Identificar tipos de columnas
columnas_numericas = X_train.select_dtypes(include=['int64', 'float64']).columns
columnas_categoricas = X_train.select_dtypes(include=['object']).columns

print(f'Variables numéricas: {len(columnas_numericas)}')
print(f'Variables categóricas: {len(columnas_categoricas)}')

# Imputar valores faltantes en numéricas
for col in columnas_numericas:
    if X_train[col].isnull().sum() > 0:
        mediana = X_train[col].median()
        X_train[col] = X_train[col].fillna(mediana)
        X_test[col] = X_test[col].fillna(mediana)

# Imputar valores faltantes en categóricas
for col in columnas_categoricas:
    if X_train[col].isnull().sum() > 0:
        moda = X_train[col].mode()[0]
        X_train[col] = X_train[col].fillna(moda)
        X_test[col] = X_test[col].fillna(moda)

print('\nValores faltantes imputados')




PASO 4: PREPROCESAMIENTO
Variables numéricas: 5
Variables categóricas: 14

Valores faltantes imputados


In [ ]:
# 5. FEATURE ENGINEERING
print('\n' + '='*60)
print(' PASO 5: FEATURE ENGINEERING')
print('='*60)

# ----- 5.1: ÍNDICE SOCIOECONÓMICO COMPUESTO -----
print('\n Creando índice socioeconómico...')

# Mapear estrato a valor numérico
estrato_map = {
    'Estrato 1': 1, 'Estrato 2': 2, 'Estrato 3': 3,
    'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6,
    'Sin Estrato': 0
}

X_train['nivel_estrato'] = X_train['F_ESTRATOVIVIENDA'].map(estrato_map).fillna(0)
X_test['nivel_estrato'] = X_test['F_ESTRATOVIVIENDA'].map(estrato_map).fillna(0)

# Score de recursos tecnológicos (0-4)
X_train['recursos_tech'] = (
    X_train['F_TIENEINTERNET'].map({'Si': 1, 'No': 0}).fillna(0) +
    X_train['F_TIENECOMPUTADOR'].map({'Si': 1, 'No': 0}).fillna(0) +
    X_train['F_TIENELAVADORA'].map({'Si': 1, 'No': 0}).fillna(0) +
    X_train['F_TIENEAUTOMOVIL'].map({'Si': 1, 'No': 0}).fillna(0)
)
X_test['recursos_tech'] = (
    X_test['F_TIENEINTERNET'].map({'Si': 1, 'No': 0}).fillna(0) +
    X_test['F_TIENECOMPUTADOR'].map({'Si': 1, 'No': 0}).fillna(0) +
    X_test['F_TIENELAVADORA'].map({'Si': 1, 'No': 0}).fillna(0) +
    X_test['F_TIENEAUTOMOVIL'].map({'Si': 1, 'No': 0}).fillna(0)
)

# Índice socioeconómico combinado
X_train['indice_socioeconomico'] = X_train['nivel_estrato'] * 0.5 + X_train['recursos_tech'] * 0.5
X_test['indice_socioeconomico'] = X_test['nivel_estrato'] * 0.5 + X_test['recursos_tech'] * 0.5

print(f'   ✓ Índice socioeconómico creado')

# ----- 5.2: EDUCACIÓN PARENTAL COMBINADA -----
print('\n Creando variables de educación parental...')

# Mapear nivel educativo a valor numérico
educacion_map = {
    'Ninguno': 0,
    'Primaria incompleta': 1,
    'Primaria completa': 2,
    'Secundaria (Bachillerato) incompleta': 3,
    'Secundaria (Bachillerato) completa': 4,
    'Técnica o tecnológica incompleta': 5,
    'Técnica o tecnológica completa': 6,
    'Educación profesional incompleta': 7,
    'Educación profesional completa': 8,
    'Postgrado': 9
}

X_train['nivel_educ_padre'] = X_train['F_EDUCACIONPADRE'].map(educacion_map).fillna(0)
X_test['nivel_educ_padre'] = X_test['F_EDUCACIONPADRE'].map(educacion_map).fillna(0)

X_train['nivel_educ_madre'] = X_train['F_EDUCACIONMADRE'].map(educacion_map).fillna(0)
X_test['nivel_educ_madre'] = X_test['F_EDUCACIONMADRE'].map(educacion_map).fillna(0)

# Promedio de educación parental
X_train['educ_parental_promedio'] = (X_train['nivel_educ_padre'] + X_train['nivel_educ_madre']) / 2
X_test['educ_parental_promedio'] = (X_test['nivel_educ_padre'] + X_test['nivel_educ_madre']) / 2

# Máximo nivel educativo parental
X_train['educ_parental_max'] = X_train[['nivel_educ_padre', 'nivel_educ_madre']].max(axis=1)
X_test['educ_parental_max'] = X_test[['nivel_educ_padre', 'nivel_educ_madre']].max(axis=1)

print(f'   ✓ Variables de educación parental creadas')

# ----- 5.3: TRABAJO Y DEDICACIÓN -----
print('\n Creando variables de trabajo...')

# Mapear horas de trabajo
trabajo_map = {
    'No trabajo': 0,
    'Menos de 10 horas': 5,
    'Entre 11 y 20 horas': 15,
    'Entre 21 y 30 horas': 25,
    'Más de 30 horas': 35
}

X_train['horas_trabajo_num'] = X_train['E_HORASSEMANATRABAJA'].map(trabajo_map).fillna(0)
X_test['horas_trabajo_num'] = X_test['E_HORASSEMANATRABAJA'].map(trabajo_map).fillna(0)

# Indicador de estudiante tiempo completo
X_train['estudiante_tiempo_completo'] = (X_train['horas_trabajo_num'] == 0).astype(int)
X_test['estudiante_tiempo_completo'] = (X_test['horas_trabajo_num'] == 0).astype(int)

# Mapear valor de matrícula
matricula_map = {
    'Menos de 500 mil': 250000,
    'Entre 500 mil y menos de 1 millón': 750000,
    'Entre 1 millón y menos de 2.5 millones': 1750000,
    'Entre 2.5 millones y menos de 4 millones': 3250000,
    'Entre 4 millones y menos de 5.5 millones': 4750000,
    'Entre 5.5 millones y menos de 7 millones': 6250000,
    'Más de 7 millones': 8000000
}

X_train['valor_matricula_num'] = X_train['E_VALORMATRICULAUNIVERSIDAD'].map(matricula_map).fillna(0)
X_test['valor_matricula_num'] = X_test['E_VALORMATRICULAUNIVERSIDAD'].map(matricula_map).fillna(0)

print(f'   ✓ Variables de trabajo y matrícula creadas')

# ----- 5.4: INDICADORES - ESTADÍSTICAS Y RATIOS -----
print('\n Creando estadísticas de indicadores...')

indicadores = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

# Promedio de indicadores
X_train['indicadores_promedio'] = X_train[indicadores].mean(axis=1)
X_test['indicadores_promedio'] = X_test[indicadores].mean(axis=1)

# Desviación estándar (variabilidad del desempeño)
X_train['indicadores_std'] = X_train[indicadores].std(axis=1)
X_test['indicadores_std'] = X_test[indicadores].std(axis=1)

# Máximo y mínimo
X_train['indicador_max'] = X_train[indicadores].max(axis=1)
X_test['indicador_max'] = X_test[indicadores].max(axis=1)

X_train['indicador_min'] = X_train[indicadores].min(axis=1)
X_test['indicador_min'] = X_test[indicadores].min(axis=1)

# Rango (diferencia max-min)
X_train['indicador_rango'] = X_train['indicador_max'] - X_train['indicador_min']
X_test['indicador_rango'] = X_test['indicador_max'] - X_test['indicador_min']

# Ratios entre indicadores
X_train['ratio_ind1_ind2'] = X_train['INDICADOR_1'] / (X_train['INDICADOR_2'] + 0.001)
X_test['ratio_ind1_ind2'] = X_test['INDICADOR_1'] / (X_test['INDICADOR_2'] + 0.001)

X_train['ratio_ind3_ind4'] = X_train['INDICADOR_3'] / (X_train['INDICADOR_4'] + 0.001)
X_test['ratio_ind3_ind4'] = X_test['INDICADOR_3'] / (X_test['INDICADOR_4'] + 0.001)

# Indicador dominante
X_train['indicador_dominante'] = X_train[indicadores].idxmax(axis=1)
X_test['indicador_dominante'] = X_test[indicadores].idxmax(axis=1)

print(f' Estadísticas de indicadores creadas')

# ----- 5.5: AGREGACIONES POR PROGRAMA Y DEPARTAMENTO (TARGET ENCODING) -----
print('\n Creando agregaciones por programa académico...')

# Target encoding por programa académico
for col in indicadores:
    # Promedio por programa
    prog_mean = X_train.groupby('E_PRGM_ACADEMICO')[col].mean()
    X_train[f'{col}_prog_mean'] = X_train['E_PRGM_ACADEMICO'].map(prog_mean)
    X_test[f'{col}_prog_mean'] = X_test['E_PRGM_ACADEMICO'].map(prog_mean)

    # Promedio por departamento
    dept_mean = X_train.groupby('E_PRGM_DEPARTAMENTO')[col].mean()
    X_train[f'{col}_dept_mean'] = X_train['E_PRGM_DEPARTAMENTO'].map(dept_mean)
    X_test[f'{col}_dept_mean'] = X_test['E_PRGM_DEPARTAMENTO'].map(dept_mean)

    # Rellenar NaN con la media global
    global_mean = X_train[col].mean()
    X_train[f'{col}_prog_mean'] = X_train[f'{col}_prog_mean'].fillna(global_mean)
    X_test[f'{col}_prog_mean'] = X_test[f'{col}_prog_mean'].fillna(global_mean)
    X_train[f'{col}_dept_mean'] = X_train[f'{col}_dept_mean'].fillna(global_mean)
    X_test[f'{col}_dept_mean'] = X_test[f'{col}_dept_mean'].fillna(global_mean)

# Conteo de estudiantes por programa
prog_counts = X_train['E_PRGM_ACADEMICO'].value_counts()
X_train['programa_frecuencia'] = X_train['E_PRGM_ACADEMICO'].map(prog_counts).fillna(0)
X_test['programa_frecuencia'] = X_test['E_PRGM_ACADEMICO'].map(prog_counts).fillna(0)

print(f' Agregaciones por programa y departamento creadas')

# ----- 5.6: INTERACCIONES CLAVE -----
print('\n Creando interacciones entre variables...')

# Interacción trabajo vs autonomía financiera
X_train['trabajo_paga_matricula'] = (
    X_train['horas_trabajo_num'].astype(str) + '_' +
    X_train['E_PAGOMATRICULAPROPIO'].astype(str)
)
X_test['trabajo_paga_matricula'] = (
    X_test['horas_trabajo_num'].astype(str) + '_' +
    X_test['E_PAGOMATRICULAPROPIO'].astype(str)
)

# Interacción recursos vs educación
X_train['recursos_vs_educacion'] = X_train['recursos_tech'] * X_train['educ_parental_promedio']
X_test['recursos_vs_educacion'] = X_test['recursos_tech'] * X_test['educ_parental_promedio']

print(f' Interacciones creadas')

print(f'\n Feature Engineering completado')
print(f'   Nuevas dimensiones: X_train {X_train.shape}, X_test {X_test.shape}')



🆕 PASO 5: FEATURE ENGINEERING AVANZADO

📊 Creando índice socioeconómico...
   ✓ Índice socioeconómico creado

📚 Creando variables de educación parental...
   ✓ Variables de educación parental creadas

💼 Creando variables de trabajo...
   ✓ Variables de trabajo y matrícula creadas

📈 Creando estadísticas de indicadores...
   ✓ Estadísticas de indicadores creadas

🎓 Creando agregaciones por programa académico...
   ✓ Agregaciones por programa y departamento creadas

🔗 Creando interacciones entre variables...
   ✓ Interacciones creadas

✅ Feature Engineering completado
   Nuevas dimensiones: X_train (692500, 48), X_test (296786, 48)


In [ ]:
# PASO 6. CODIFICACIÓN DE VARIABLES CATEGÓRICAS
print('\n' + '='*60)
print('PASO 6: CODIFICACIÓN DE VARIABLES CATEGÓRICAS')
print('='*60)

# Identificar columnas categóricas
columnas_categoricas = X_train.select_dtypes(include=['object']).columns.tolist()
print(f'Variables categóricas a codificar: {len(columnas_categoricas)}')

# Label Encoding para todas las categóricas
label_encoders = {}
for col in columnas_categoricas:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))
    X_test[col] = X_test[col].astype(str).map(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
    label_encoders[col] = le

print('Variables categóricas codificadas ✓')



PASO 6: CODIFICACIÓN DE VARIABLES CATEGÓRICAS
Variables categóricas a codificar: 16
Variables categóricas codificadas ✓


In [ ]:
# PASO 7. PREPARAR TRAIN/VALIDATION SPLIT
print('\n' + '='*60)
print('PASO 7: DIVISIÓN TRAIN/VALIDATION')
print('='*60)

# Codificar variable objetivo
le_target = LabelEncoder()
y_train_encoded = le_target.fit_transform(y_train)

# Split
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_train_encoded
)

print(f'Train split: {X_train_split.shape}')
print(f'Validation split: {X_val_split.shape}')



PASO 7: DIVISIÓN TRAIN/VALIDATION
Train split: (554000, 48)
Validation split: (138500, 48)


In [ ]:
# PASO 8: OPTIMIZACIÓN DE PARÁMETROS
print('\n' + '='*60)
print('PASO 8: CONFIGURACIÓN DEL MODELO')
print('='*60)
print('\n Usando parámetros optimizados manualmente...')

mejor_modelo = XGBClassifier(
    n_estimators=500,
    max_depth=9,
    learning_rate=0.08,
    subsample=0.85,
    colsample_bytree=0.85,
    gamma=0.15,
    min_child_weight=2,
    reg_alpha=0.2,
    reg_lambda=1.2,
    random_state=42,
    eval_metric='mlogloss',
    n_jobs=-1
)


PASO 8: CONFIGURACIÓN DEL MODELO

📊 Usando hiperparámetros optimizados manualmente...


In [ ]:
# PASO 9. ENTRENAMIENTO Y EVALUACIÓN
print('\n' + '='*60)
print('PASO 9: ENTRENAMIENTO Y EVALUACIÓN')
print('='*60)

print('\n Entrenando modelo...')
mejor_modelo.fit(X_train_split, y_train_split)
print('Modelo entrenado')

# Evaluar
y_pred_val = mejor_modelo.predict(X_val_split)
y_pred_val_decoded = le_target.inverse_transform(y_pred_val)
y_val_decoded = le_target.inverse_transform(y_val_split)

accuracy = accuracy_score(y_val_decoded, y_pred_val_decoded)
print(f'\n Accuracy en validación: {accuracy:.4f} ({accuracy*100:.2f}%)')
print('\n Reporte de clasificación:')
print(classification_report(y_val_decoded, y_pred_val_decoded))


PASO 9: ENTRENAMIENTO Y EVALUACIÓN

🚀 Entrenando modelo...
   ✓ Modelo entrenado

🎯 Accuracy en validación: 0.4369 (43.69%)

📊 Reporte de clasificación:
              precision    recall  f1-score   support

        alto       0.56      0.63      0.59     35124
        bajo       0.47      0.55      0.51     34597
  medio-alto       0.33      0.28      0.30     34324
  medio-bajo       0.33      0.28      0.31     34455

    accuracy                           0.44    138500
   macro avg       0.42      0.44      0.43    138500
weighted avg       0.42      0.44      0.43    138500



In [ ]:
# PASO 10. PSEUDO-LABELING
print('\n' + '='*60)
print(' PASO 10: PSEUDO-LABELING')
print('='*60)
print('\n Aplicando pseudo-labeling...')

# Predecir en test con probabilidades
predicciones_test_proba = mejor_modelo.predict_proba(X_test)
max_proba = predicciones_test_proba.max(axis=1)

# Seleccionar solo predicciones con alta confianza (>95%)
umbral_confianza = 0.95
indices_confiables = max_proba > umbral_confianza
X_test_confiable = X_test[indices_confiables]
y_test_pseudo = mejor_modelo.predict(X_test_confiable)

print(f'Datos con pseudo-labels (confianza >{umbral_confianza}): {len(X_test_confiable)} ({len(X_test_confiable)/len(X_test)*100:.1f}%)')

if len(X_test_confiable) > 100:  # Solo si hay suficientes datos confiables
    # Combinar con datos de entrenamiento
    X_train_extended = pd.concat([X_train, X_test_confiable], ignore_index=True)
    y_train_extended = np.concatenate([y_train_encoded, y_test_pseudo])

    # Re-entrenar
    print('Re-entrenando con datos extendidos...')
    mejor_modelo.fit(X_train_extended, y_train_extended)
    print(' Modelo re-entrenado con pseudo-labeling')
else:
    print('No hay suficientes datos confiables para pseudo-labeling')



🆕 PASO 10: PSEUDO-LABELING (OPCIONAL)

🔄 Aplicando pseudo-labeling...
   Datos con pseudo-labels (confianza >0.95): 477 (0.2%)
   🔄 Re-entrenando con datos extendidos...
   ✓ Modelo re-entrenado con pseudo-labeling


In [ ]:
# PASO 11: MODELO FINAL CON TODOS LOS DATOS
print('\n' + '='*60)
print('PASO 11: ENTRENAMIENTO DEL MODELO FINAL')
print('='*60)

print('\n Entrenando modelo final con todos los datos de entrenamiento')

modelo_final = XGBClassifier(
    n_estimators=700,
    max_depth=9,
    learning_rate=0.08,
    subsample=0.85,
    colsample_bytree=0.85,
    gamma=0.15,
    min_child_weight=2,
    reg_alpha=0.2,
    reg_lambda=1.2,
    random_state=42,
    eval_metric='mlogloss',
    n_jobs=-1
)

modelo_final.fit(X_train, y_train_encoded)
print(' Modelo final entrenado')



PASO 11: ENTRENAMIENTO DEL MODELO FINAL

🎓 Entrenando modelo final con todos los datos de entrenamiento...
✅ Modelo final entrenado


In [ ]:
# PASO 12: PREDICCIONES EN TEST

print('\n' + '='*60)
print('PASO 12: PREDICCIONES EN TEST')
print('='*60)

predicciones_encoded = modelo_final.predict(X_test)
predicciones = le_target.inverse_transform(predicciones_encoded)

print(f'\n Predicciones realizadas: {len(predicciones)}')
print('\n Distribución de predicciones:')
print(pd.Series(predicciones).value_counts().sort_index())



PASO 12: PREDICCIONES EN TEST

✅ Predicciones realizadas: 296786

📊 Distribución de predicciones:
alto          83192
bajo          88073
medio-alto    62555
medio-bajo    62966
Name: count, dtype: int64


In [ ]:
# PASO 13: CREAR ARCHIVO SUBMISSION
print('\n' + '='*60)
print('PASO 13: GENERAR ARCHIVO DE SUBMISSION')
print('='*60)

submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': predicciones
})

submission.to_csv('submission.csv', index=False)

print(f'\n Archivo creado: submission.csv')
print(f'   Total de predicciones: {len(submission)}')
print('\n Primeras 10 predicciones:')
print(submission.head(10))

print('\n' + '='*60)
print(' PROCESO COMPLETADO')
print('='*60)
print(f' Accuracy esperado en validación: {accuracy:.4f}')


PASO 13: GENERAR ARCHIVO DE SUBMISSION

✅ Archivo creado: submission.csv
   Total de predicciones: 296786

📋 Primeras 10 predicciones:
       ID RENDIMIENTO_GLOBAL
0  550236               bajo
1   98545         medio-alto
2  499179               alto
3  782980               bajo
4  785185               bajo
5   58495               bajo
6  705444               alto
7  557548               alto
8  519909               bajo
9  832058               alto

🎉 PROCESO COMPLETADO
📁 Descarga submission_mejorado.csv y súbelo a Kaggle
🎯 Accuracy esperado en validación: 0.4369

💡 Mejoras implementadas:
   ✓ Feature Engineering avanzado (+40 nuevas variables)
   ✓ Índice socioeconómico compuesto
   ✓ Educación parental agregada
   ✓ Target encoding por programa/departamento
   ✓ Estadísticas de indicadores
   ✓ Interacciones clave
   ✓ Hiperparámetros optimizados

🚀 Para mejorar aún más:
   1. Descomenta la sección de Optuna para optimización automática
   2. Descomenta la sección de Pseudo-Labelin